In [85]:
# Recode YouGov File to match ANES
#including changing ANES predictor column names
#and data from both files

In [86]:
import pandas as pd
import numpy as np


df_yougov_filepath = r"C:\Users\kirin\Downloads\W1_W2_W3_Merged_saved (1).dta" # raw .dta yougov file
df_anes_filepath = r"C:\Users\kirin\Downloads\anes2020\anes_timeseries_2020_stata_20220210.dta" # raw ANES 2020 timeseries

df_yougov = pd.read_stata(df_yougov_filepath, convert_categoricals=True) # categorical data for easy recoding
df_yougov_nums = pd.read_stata(df_yougov_filepath, convert_categoricals=False) # we set a few columns to numeric to allow for easier code matching. 

df_yougov['abortion'] = df_yougov_nums['abortion'] ### adding abortion
mapping = {
    4.0 : 1.0,
    3.0 : 2.0,
    2.0 : 3.0,
    1.0 : 4.0
}

df_anes = pd.read_stata(df_anes_filepath, convert_categoricals=False)
### pre election variables: start with V201
post_col_names = {'V202242x', 'V202245x', 'V202266', 'V202267', 'V202268', 'V202468x'} ## add predictors from post election . Remove these if not enough data
mask = df_anes.columns.str.startswith("V201") | df_anes.columns.isin(post_col_names) # keep pre election plus extra vars
df_anes = df_anes.loc[:, mask]

df_anes[df_anes < 0] = np.nan #negative numbers are used to signify categories like "refused" or "inapplicable"
df_anes[(df_anes == 998) | (df_anes == 999) | (df_anes == 99)] = np.nan # 998/999 are don't know how to respond and 99 is no response


In [87]:
### todo: Racial Resentment, Moral Traditionalism, Republican Party / Democratic Party
df_anes = df_anes.rename(columns={#'V201028': 'turnout20post', 
                                  #'V201029': 'presvote20post',
                                  #'V201039': 'housevote20post',
                                  #'V201049': 'senvote20post',
                                  #'V201128': 'trump_presidential_approval',
                                  'V201151': 'pff_jb',
                                  'V201152': 'pff_dt',
                                  'V201200': 'ideo7',
                                  'V201231x': 'pid7',
                                  #'V201234': 'para_social_grid_2',
                                  'V201351': 'election_fairnness',
                                 # 'V201370': 'cab_b', #w123
                                  'V201510': 'educ',
                                  'V201549x': 'race', 
                                  'V201558x': 'hispanic',
                                  'V201602': 'partisan_violence',
                                  'V201336': 'abortion',
                                  'V202242x': 'immigrant_citizenship',
                                  'V202245x': 'immigrant_deport',
                                  'V202266': 'auth_grid_1',
                                  'V202267': 'auth_grid_3',
                                  'V202268': 'auth_grid_2', # potentially incorrect numbering
                                  'V202468x': 'faminc_new' #reordered
                                  #'V202483': 'wc_together', # all of these are separated by wave
                                 # 'V202487': 'wc_jobs',
                                 # 'V202516': 'hardworkingvlazy',
                                  #'V202522': 'pronenot_violence',
                                  #'V202527': 'group_disc_black',
                                  #'V202528': 'group_disc_hispanic',
                                  #'V202530': 'group_disc_white'
                                  })
combined = [
                   #'christian_nationalism',
                   #'turnout20post', 
                   #'presvote20post',
                   #'housevote20post',
                   #'senvote20post',
                   #'trump_presidential_approval',
                   'pff_jb',
                   'pff_dt',
                   'ideo7',
                   'pid7',
                  # 'para_social_grid_2',
                   'election_fairnness',
                   #'cab_b', w1, w2, w3
                   'educ',
                   'race',
                   'hispanic',
                   'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new',
                   'abortion'
                  # 'wc_together', w1, w2, w3
                   #'wc_jobs',
                   #'racial_id',
                   #'hardworkingvlazy',
                   #'pronenot_violence',
                   #'group_disc_black',
                   #'group_disc_hispanic',
                   #'group_disc_white'
                   ] 
#df_yougov = df_yougov[combined] # disable this to generate test set
df_anes['abortion'] = df_anes['abortion'].map(mapping).fillna(np.nan).astype('float') 

# Count the number of 1.0s and 2.0s
counts = df_yougov['abortion'].value_counts()

# Print the counts for 1.0 and 2.0
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))

counts = df_anes['abortion'].value_counts()
# Print the counts for 1.0 and 2.0
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('1: ', counts.get(3.0, 0))
print('2: ', counts.get(4.0, 0))


1:  1154
2:  301
3:  656
4:  331
1:  4017
2:  1097
1:  1916
2:  870


In [88]:
#import re 
# function to extract the number from the string
#def extract_number(entry):
#    if isinstance(entry, str):
#        match = re.search(r'-?\d+(\.\d+)?', entry)
#        if match:
#            return float(match.group())  # Convert matched number to float
#    elif isinstance(entry, (int, float)):
#        return entry  # In case the entry already a float or int, return it as is
#    return np.nan # if not a number , nan
#
# Apply function to all relevant columns
#for column in df_anes.columns:
#    df_anes[column] = df_anes[column].apply(extract_number).astype(float, errors='ignore')

In [89]:
'''print(df_anes['turnout20post'].unique())'''

"print(df_anes['turnout20post'].unique())"

In [90]:
#RACE
#1. White, non-Hispanic
#2. Black, non-Hispanic
#3. Hispanic
#4. Asian or Native Hawaiian/other Pacific Islander, non-Hispanic
#alone


# Create the 'white' column based on the 'race' column
df_yougov['white'] = df_yougov['race'].apply(lambda x: 1 if x == 'White' else 0)

In [91]:
print(df_anes['race'])

0       3.0
1       4.0
2       1.0
3       4.0
4       5.0
       ... 
8275    1.0
8276    1.0
8277    3.0
8278    1.0
8279    4.0
Name: race, Length: 8280, dtype: float64


In [92]:
df_anes['white'] = df_anes['race'].apply(lambda x: 1 if x == 1.0 else 0)

In [93]:
#turnout20post = V201028
# ANES: 
#-9 refused
# -1 inapplicable
# 1 Yes
# 2 No

mapping = {
    'Yes': 1.0,
    'No': 2.0
}

# Apply the mapping to the yougov column, using NaN for other entries
#df_yougov['turnout20post'] = df_yougov['turnout20post'].map(mapping).fillna(np.nan).astype('float')

In [94]:
# Count the number of 1.0s and 2.0s
#counts = df_yougov['turnout20post'].value_counts()

# Print the counts for 1.0 and 2.0
#print('1: ', counts.get(1.0, 0))
#print('2: ', counts.get(2.0, 0))
#print(df_yougov['turnout20post'].unique())

In [95]:
# now recode ANES
#print(df_anes['turnout20post'].unique())
#already done; nan, 1, 2

In [96]:
#presvote20post = V201029
#-9. Refused
#-1. Inapplicable
#1. Joe Biden
#2. Donald Trump
#3. Jo Jorgensen
#4. Howie Hawkins
#5. Other candidate {SPECIFY}
#12. Specified as refused

#remapping biden = 1 trump = 2
'''
mapping = {
    'Joe Biden': 1.0,
    'Donald Trump': 2.0,
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['presvote20post'] = df_yougov['presvote20post'].map(mapping).fillna(np.nan).astype('float')'
'''

"\nmapping = {\n    'Joe Biden': 1.0,\n    'Donald Trump': 2.0,\n}\n# Apply the mapping to the yougov column, using NaN for other entries\ndf_yougov['presvote20post'] = df_yougov['presvote20post'].map(mapping).fillna(np.nan).astype('float')'\n"

In [97]:
'''print(df_yougov['presvote20post'].unique())'''

"print(df_yougov['presvote20post'].unique())"

In [98]:
'''df_anes.loc[~df_anes['presvote20post'].isin([1.0, 2.0]), 'presvote20post'] = np.nan'''

"df_anes.loc[~df_anes['presvote20post'].isin([1.0, 2.0]), 'presvote20post'] = np.nan"

In [99]:
'''print(df_anes['presvote20post'].unique())'''

"print(df_anes['presvote20post'].unique())"

In [100]:
'''### housevote20post = V201039
# map 1 to democrat 2 to republican 
mapping = {
    'Yes, I voted for the Republican candidate': 2.0,
    'Yes, I voted for the Democratic candidate': 1.0,
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['housevote20post'] = df_yougov['housevote20post'].map(mapping).fillna(np.nan).astype('float')'''

"### housevote20post = V201039\n# map 1 to democrat 2 to republican \nmapping = {\n    'Yes, I voted for the Republican candidate': 2.0,\n    'Yes, I voted for the Democratic candidate': 1.0,\n}\n# Apply the mapping to the yougov column, using NaN for other entries\ndf_yougov['housevote20post'] = df_yougov['housevote20post'].map(mapping).fillna(np.nan).astype('float')"

In [101]:
'''print(df_yougov['housevote20post'].unique())'''

"print(df_yougov['housevote20post'].unique())"

In [102]:
'''df_anes.loc[~df_anes['housevote20post'].isin([1.0, 2.0]), 'housevote20post'] = np.nan'''

"df_anes.loc[~df_anes['housevote20post'].isin([1.0, 2.0]), 'housevote20post'] = np.nan"

In [103]:
'''print(df_anes['housevote20post'].unique())'''

"print(df_anes['housevote20post'].unique())"

In [104]:
'''###senvote20post = V201049
# 1 dem 2 rep

mapping = {
    'Yes, I voted for the Republican candidate': 2.0,
    'Yes, I voted for the Democratic candidate': 1.0,
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['senvote20post'] = df_yougov['senvote20post'].map(mapping).fillna(np.nan).astype('float')'''

"###senvote20post = V201049\n# 1 dem 2 rep\n\nmapping = {\n    'Yes, I voted for the Republican candidate': 2.0,\n    'Yes, I voted for the Democratic candidate': 1.0,\n}\n# Apply the mapping to the yougov column, using NaN for other entries\ndf_yougov['senvote20post'] = df_yougov['senvote20post'].map(mapping).fillna(np.nan).astype('float')"

In [105]:
'''print(df_yougov['senvote20post'].unique())'''

"print(df_yougov['senvote20post'].unique())"

In [106]:
'''df_anes.loc[~df_anes['senvote20post'].isin([1.0, 2.0]), 'senvote20post'] = np.nan'''

"df_anes.loc[~df_anes['senvote20post'].isin([1.0, 2.0]), 'senvote20post'] = np.nan"

In [107]:
'''print(df_anes['senvote20post'].unique())'''

"print(df_anes['senvote20post'].unique())"

In [108]:
### pff_jb = V201151

mapping = {
    'Very unfavorable': 5.0, ### reverse
    'Somewhat unfavorable': 4.0,
    'Neutral': 3.0,
    'Somewhat favorable': 2.0,
    'Very favorable': 1.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['pff_jb'] = df_yougov['pff_jb'].map(mapping).fillna(np.nan).astype('float')


In [109]:
# Count the number of 1.0s and 2.0s
counts = df_yougov['pff_jb'].value_counts()

# Print the counts for 1.0 and 2.0
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))


1:  479
2:  430
3:  362
4:  244
5:  902


In [110]:
#anes feeling thermom

# Convert all entries to numeric, setting errors='coerce' will replace non-numeric values with NaN
df_anes['pff_jb'] = pd.to_numeric(df_anes['pff_jb'], errors='coerce')

# Replace entries that are not between 0 and 100 (inclusive) with NaN
df_anes.loc[~((df_anes['pff_jb'] >= 0) & (df_anes['pff_jb'] <= 100)), 'pff_jb'] = np.nan
df_anes['pff_jb'] = 100 - df_anes['pff_jb']

In [111]:
print(df_anes['pff_jb'].unique())

[100.  35.  30.  85.  15.  50.   0.  40.  25.  60.  20.  70.  28.  nan
  80.  10.   5.  45.  16.  65.   3.  90.  42.   4.  14.  38.  95.  99.
  75.  49.  91.  39.  13.  51.  98.  52.  23.  34.  87.  97.  11.  55.
  96.  12.   2.  33.  89.  94.   7.  67.  69.  17.  48.  74.  88.  37.
  27.  93.  64.  31.  71.]


In [112]:
### V201152 = pff_dt
mapping = {
    'Very unfavorable': 1.0,
    'Somewhat unfavorable': 2.0,
    'Neutral': 3.0,
    'Somewhat favorable': 4.0,
    'Very favorable': 5.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['pff_dt'] = df_yougov['pff_dt'].map(mapping).fillna(np.nan).astype('float')

In [113]:
# Count the number of 1.0s and 2.0s
counts = df_yougov['pff_dt'].value_counts()

# Print the counts for 1.0 and 2.0
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))


1:  1111
2:  176
3:  283
4:  310
5:  535


In [114]:
#anes feeling thermom

# Convert all entries to numeric, setting errors='coerce' will replace non-numeric values with NaN
df_anes['pff_dt'] = pd.to_numeric(df_anes['pff_dt'], errors='coerce')

# Replace entries that are not between 0 and 100 (inclusive) with NaN
df_anes.loc[~((df_anes['pff_dt'] >= 0) & (df_anes['pff_dt'] <= 100)), 'pff_dt'] = np.nan

In [115]:
print(df_anes['pff_dt'].unique())

[100.   0.  15.  85.  75.  40.  95.   5.  60.  nan  70.  98.  80.  30.
  65.  50.  90.  10.  20.  18.  68.   3.  66.  52.  25.  55.  86.  45.
   9.   1.  89.  87.   8.   7.  88.  35.   6.  24.  69.  74.  92.  49.
   2.  72.  77.  34.  12.  94.  91.  82.  29.  56.  51.  64.]


In [116]:
### V201200 = ideo7
### 1 liberal to 7 conservative 
### V201152 = pff_dt
mapping = {
    'Extremely liberal': 1.0,
    'Liberal': 2.0,
    'Slightly liberal': 3.0,
    'Moderate; middle of the road': 4.0,
    'Slightly conservative': 5.0,
    'Conservative': 6.0,
    'Extremely conservative': 7.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['ideo7'] = df_yougov['ideo7'].map(mapping).fillna(np.nan).astype('float')

In [117]:
# Count the number of 1.0s and 2.0s
counts = df_yougov['ideo7'].value_counts()

# Print the counts for 1.0 and 2.0
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))
print('6: ', counts.get(6.0, 0))
print('7: ', counts.get(7.0, 0))


1:  202
2:  404
3:  159
4:  811
5:  190
6:  443
7:  233


In [118]:
df_anes.loc[~df_anes['ideo7'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]), 'ideo7'] = np.nan

In [119]:
print(df_anes['ideo7'].unique())

[ 6.  4.  2.  3.  5. nan  1.  7.]


In [120]:
### pid7 = V201231x
#1 dem to 7 republican
df_anes.loc[~df_anes['pid7'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]), 'pid7'] = np.nan
print(df_anes['pid7'].unique())

[ 7.  4.  3.  6.  2.  1.  5. nan]


In [121]:

mapping = {
    'Strong Democrat': 1.0,
    'Not very strong Democrat': 2.0,
    'Lean Democrat': 3.0,
    'Independent': 4.0,
    'Lean Republican': 5.0,
    'Not very strong Republican': 6.0,
    'Strong Republican': 7.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['pid7'] = df_yougov['pid7'].map(mapping).fillna(np.nan).astype('float')

In [122]:
counts = df_yougov['pid7'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))
print('6: ', counts.get(6.0, 0))
print('7: ', counts.get(7.0, 0))

1:  567
2:  286
3:  203
4:  456
5:  219
6:  203
7:  418


In [123]:
'''# V201351 = election_fairnness
#anes 1 not confident to 5 confident

df_anes.loc[~df_anes['election_fairnness'].isin([1.0, 2.0, 3.0, 4.0, 5.0]), 'election_fairnness'] = np.nan
print(df_anes['election_fairnness'].unique())'''


"# V201351 = election_fairnness\n#anes 1 not confident to 5 confident\n\ndf_anes.loc[~df_anes['election_fairnness'].isin([1.0, 2.0, 3.0, 4.0, 5.0]), 'election_fairnness'] = np.nan\nprint(df_anes['election_fairnness'].unique())"

In [124]:
'''
mapping = {
    'Extremely confident': 5.0,
    'Very confident': 4.0,
    'Moderately confident': 3.0,
    'A little confident': 2.0,
    'Not at all confident': 1.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['election_fairnness'] = df_yougov['election_fairnness'].map(mapping).fillna(np.nan).astype('float')'''

"\nmapping = {\n    'Extremely confident': 5.0,\n    'Very confident': 4.0,\n    'Moderately confident': 3.0,\n    'A little confident': 2.0,\n    'Not at all confident': 1.0\n}\n# Apply the mapping to the yougov column, using NaN for other entries\ndf_yougov['election_fairnness'] = df_yougov['election_fairnness'].map(mapping).fillna(np.nan).astype('float')"

In [125]:
'''counts = df_yougov['election_fairnness'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))'''

"counts = df_yougov['election_fairnness'].value_counts()\n# Print the counts \nprint('1: ', counts.get(1.0, 0))\nprint('2: ', counts.get(2.0, 0))\nprint('3: ', counts.get(3.0, 0))\nprint('4: ', counts.get(4.0, 0))\nprint('5: ', counts.get(5.0, 0))"

In [126]:
### V201510 = educ
# ANES : 
# 1 less than hs
# 2. high school
#3. some college no degree
#4. associate degree in college - occupational/vocational
#5. Associate degree in college - academic
# 6. bachelor's degree 
#7. Master's degree
# 8. Professional/doctorate degree

df_anes.loc[~df_anes['educ'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]), 'educ'] = np.nan
mapping = { # reverse coding so liberal (high education) is low
    1.0: 6.0,
    2.0: 5.0,
    3.0: 4.0,
    4.0: 3.0,
    5.0: 2.0,
    6.0: 1.0
}
df_anes['educ'] = df_anes['educ'].map(mapping).fillna(np.nan).astype('float')
print(df_anes['educ'].unique())

[ 1.  4.  5.  3. nan  6.  2.]


In [127]:
#YG: indirect mapping where 1 is no educ and 6 is high

mapping = { # reverse coding so liberal (high education) is low
    'No HS': 6.0,
    'High school graduate': 5.0,
    'Some college': 4.0,
    '2-year': 3.0,
    '4-year': 2.0,
    'Post-grad': 1.0
}
# Apply the mapping to the yougov column, using NaN for other entries
df_yougov['educ'] = df_yougov['educ'].map(mapping).fillna(np.nan).astype('float')

In [128]:
'''counts = df_yougov['educ'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))'''

"counts = df_yougov['educ'].value_counts()\n# Print the counts \nprint('1: ', counts.get(1.0, 0))\nprint('2: ', counts.get(2.0, 0))\nprint('3: ', counts.get(3.0, 0))\nprint('4: ', counts.get(4.0, 0))\nprint('5: ', counts.get(5.0, 0))"

In [129]:
'''### V201558x = hispanic
# recoding all to 1. yes 2. no

mapping = {
    1.0: 1.0,
    2.0: 1.0,
    3.0: 1.0,
    4.0: 1.0,
    7.0: 2.0
}
df_anes['hispanic'] = df_anes['hispanic'].map(mapping).fillna(np.nan)
df_anes.loc[~df_anes['hispanic'].isin([1.0, 2.0]), 'hispanic'] = np.nan
print(df_anes['hispanic'].unique())'''


"### V201558x = hispanic\n# recoding all to 1. yes 2. no\n\nmapping = {\n    1.0: 1.0,\n    2.0: 1.0,\n    3.0: 1.0,\n    4.0: 1.0,\n    7.0: 2.0\n}\ndf_anes['hispanic'] = df_anes['hispanic'].map(mapping).fillna(np.nan)\ndf_anes.loc[~df_anes['hispanic'].isin([1.0, 2.0]), 'hispanic'] = np.nan\nprint(df_anes['hispanic'].unique())"

In [130]:
'''mapping = {
    'Yes': 1.0,
    'No': 2.0
}
#yougov column
df_yougov['hispanic'] = df_yougov['hispanic'].map(mapping).fillna(np.nan).astype(float)'''

"mapping = {\n    'Yes': 1.0,\n    'No': 2.0\n}\n#yougov column\ndf_yougov['hispanic'] = df_yougov['hispanic'].map(mapping).fillna(np.nan).astype(float)"

In [131]:
'''counts = df_yougov['hispanic'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))'''

"counts = df_yougov['hispanic'].value_counts()\n# Print the counts \nprint('1: ', counts.get(1.0, 0))\nprint('2: ', counts.get(2.0, 0))\nprint('3: ', counts.get(3.0, 0))"

In [132]:
'''# partisan_violence = V201602
# anes 1 not at all to 5 a great deal
df_anes.loc[~df_anes['partisan_violence'].isin([1.0, 2.0, 3.0, 4.0, 5.0]), 'partisan_violence'] = np.nan
print(df_anes['partisan_violence'].unique())'''


"# partisan_violence = V201602\n# anes 1 not at all to 5 a great deal\ndf_anes.loc[~df_anes['partisan_violence'].isin([1.0, 2.0, 3.0, 4.0, 5.0]), 'partisan_violence'] = np.nan\nprint(df_anes['partisan_violence'].unique())"

In [133]:
'''mapping = {
    'Always': 1.0,
    'Frequently': 2.0,
    'Occasionally': 3.0,
    'Never': 4.0
}
# yougov column
df_yougov['partisan_violence'] = df_yougov['partisan_violence'].map(mapping).fillna(np.nan).astype(float)'''

"mapping = {\n    'Always': 1.0,\n    'Frequently': 2.0,\n    'Occasionally': 3.0,\n    'Never': 4.0\n}\n# yougov column\ndf_yougov['partisan_violence'] = df_yougov['partisan_violence'].map(mapping).fillna(np.nan).astype(float)"

In [134]:
'''counts = df_yougov['partisan_violence'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))'''

"counts = df_yougov['partisan_violence'].value_counts()\n# Print the counts \nprint('1: ', counts.get(1.0, 0))\nprint('2: ', counts.get(2.0, 0))\nprint('3: ', counts.get(3.0, 0))\nprint('4: ', counts.get(4.0, 0))"

In [135]:
### V202242x = immigrant_citizenship
# 1 favor to 7 oppose

df_anes.loc[~df_anes['immigrant_citizenship'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]), 'immigrant_citizenship'] = np.nan
mapping = {
    7.0: 1.0,
    6.0: 2.0,
    5.0: 3.0,
    4.0: 4.0,
    3.0: 5.0,
    2.0: 6.0,
    1.0: 7.0
}
df_anes['immigrant_citizenship'] = df_anes['immigrant_citizenship'].map(mapping).fillna(np.nan).astype(float)
print(df_anes['immigrant_citizenship'].unique())

[ 7.  5.  6.  1.  4.  2.  3. nan]


In [136]:
mapping = {
    'Favor a great deal': 7.0,
    'Favor a moderate amount': 6.0,
    'Favor a little': 5.0,
    'Neither favor nor oppose': 4.0,
    'Oppose a little': 3.0,
    'Oppose a moderate amount': 2.0,
    'Oppose a great deal': 1.0
}
#yougov column
df_yougov['immigrant_citizenship'] = df_yougov['immigrant_citizenship'].map(mapping).fillna(np.nan).astype(float)

In [137]:
counts = df_yougov['immigrant_citizenship'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))
print('6: ', counts.get(6.0, 0))
print('7: ', counts.get(7.0, 0))

1:  417
2:  159
3:  131
4:  446
5:  295
6:  337
7:  657


In [138]:
### immigrant_deport = V202245x
# 1 favor to 7 oppose

df_anes.loc[~df_anes['immigrant_deport'].isin([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]), 'immigrant_deport'] = np.nan
mapping = {
    1.0: 7.0, ### reverse order
    2.0: 6.0,
    3.0: 5.0,
    4.0: 4.0,
    5.0: 3.0,
    6.0: 2.0,
    7.0: 1.0
}
df_anes['immigrant_deport'] = df_anes['immigrant_deport'].map(mapping).fillna(np.nan).astype(float)
print(df_anes['immigrant_deport'].unique())

[ 7.  4.  1.  6.  2.  3. nan  5.]


In [139]:
mapping = {
    'Favor a great deal': 1.0, ### reverse order
    'Favor a moderate amount': 2.0,
    'Favor a little': 3.0,
    'Neither favor nor oppose': 4.0,
    'Oppose a little': 5.0,
    'Oppose a moderate amount': 6.0,
    'Oppose a great deal': 7.0
}
#yougov column
df_yougov['immigrant_deport'] = df_yougov['immigrant_deport'].map(mapping).fillna(np.nan).astype(float)

In [140]:
# NOTE: suspicious numbers differ from logged yougov data
counts = df_yougov['immigrant_deport'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))
print('6: ', counts.get(6.0, 0))
print('7: ', counts.get(7.0, 0))

1:  408
2:  207
3:  199
4:  536
5:  215
6:  233
7:  644


In [141]:
# V202266 =auth_grid_1
# independence 1 respect for elders 2

df_anes.loc[~df_anes['auth_grid_1'].isin([1.0, 2.0]), 'auth_grid_1'] = np.nan
print(df_anes['auth_grid_1'].unique())

[ 2.  1. nan]


In [142]:
mapping = {
    'Independence': 1.0,
    'Respect for Elders': 2.0
}
#yougov column
df_yougov['auth_grid_1'] = df_yougov['auth_grid_1'].map(mapping).fillna(np.nan).astype(float)

In [143]:
counts = df_yougov['auth_grid_1'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))

1:  883
2:  1559


In [144]:
### V202267= auth_grid_3
#1. curiosity 2. good manners

df_anes.loc[~df_anes['auth_grid_3'].isin([1.0, 2.0]), 'auth_grid_3'] = np.nan

print(df_anes['auth_grid_3'].unique())


[ 2.  1. nan]


In [145]:
mapping = {
    'Curiosity': 1.0,
    'Good Manners': 2.0
}
#yougov column
df_yougov['auth_grid_3'] = df_yougov['auth_grid_3'].map(mapping).fillna(np.nan).astype(float)

In [146]:
counts = df_yougov['auth_grid_3'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
counts = df_anes['auth_grid_3'].value_counts()
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))

1:  998
2:  1444
1:  3242
2:  4092


In [147]:
### V202268 = auth_grid_2
#1. obedience 2. self-reliance

df_anes.loc[~df_anes['auth_grid_2'].isin([1.0, 2.0]), 'auth_grid_2'] = np.nan
mapping = {
    2.0 : 1.0,
    1.0 : 2.0
}
#yougov column
df_anes['auth_grid_2'] = df_anes['auth_grid_2'].map(mapping).fillna(np.nan).astype(float)

print(df_anes['auth_grid_2'].unique())

[ 1.  2. nan]


In [148]:
mapping = {
    'Obedience': 1.0,
    'Self-reliance': 2.0
}
#yougov column
df_yougov['auth_grid_2'] = df_yougov['auth_grid_2'].map(mapping).fillna(np.nan).astype(float)

In [149]:
#DEBUG: answers appear to be reversed 
counts = df_yougov['auth_grid_2'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))

counts = df_anes['auth_grid_2'].value_counts()
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))

1:  1575
2:  867
1:  4357
2:  2982


In [150]:
### faminc_new = V202468x
# ANES: 1 low through 22 high (250000)

# Convert all entries to numeric, setting errors='coerce' will replace non-numeric values with NaN
df_anes['faminc_new'] = pd.to_numeric(df_anes['faminc_new'], errors='coerce')

# Replace entries that are not between 0 and 100 (inclusive) with NaN
df_anes.loc[~((df_anes['faminc_new'] >= 0) & (df_anes['faminc_new'] <= 22)), 'faminc_new'] = np.nan

print(df_anes['faminc_new'].unique())


[21. 13. 17.  7. 22.  3.  4. 10. 11.  9. 18.  1. 20.  5. 12.  6. 15.  8.
 nan 19. 14. 16.  2.]


In [151]:
mapping = { # recoding to match ANES data
    'Less than $10,000': 1.0,
    '$10,000 - $19,999': 2.5,
    '$20,000 - $29,999': 4.5,
    '$30,000 - $39,999': 6.5,
    '$40,000 - $49,999': 8.5,
    '$50,000 - $59,999': 10.0,
    '$60,000 - $69,999': 11.5,
    '$70,000 - $79,999': 13.5,
    '$80,000 - $99,999': 15.5,
    '$100,000 - $119,999': 17.5,
    '$120,000 - $149,999': 18.83,
    '$150,000 - $199,999': 20.5,
    '$200,000 - $249,999': 21.0,
    '$250,000 - $349,999': 22.0,
    '$350,000 - $499,999': 22.0,
    '$500,000 or more': 22.0
}
# yougov column
df_yougov['faminc_new'] = df_yougov['faminc_new'].map(mapping).fillna(np.nan).astype(float)

In [152]:
# DEBUG: this doesn't look right either
counts = df_yougov['faminc_new'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('12: ', counts.get(12.0, 0))

1:  181
2:  0
12:  0


In [153]:
### RACE RESENT

In [154]:
#DEBUG: w1, w2, w3 specifications
### wc_together = V202483
# 1 important to 5 not
#df_anes.loc[~df_anes['wc_together'].isin([1.0, 2.0, 3.0, 4.0, 5.0]), 'wc_together'] = np.nan
#print(df_anes['wc_together'].unique())

In [155]:
'''mapping = {
    'Extremely important': 1.0,
    'Very important': 2.0,
    'Moderately important': 3.0,
    'A little important': 4.0,
    'Not at all important': 5.0
}
df_yougov['wc_together'] = df_yougov['wc_together'].map(mapping).fillna(np.nan)
'''

"mapping = {\n    'Extremely important': 1.0,\n    'Very important': 2.0,\n    'Moderately important': 3.0,\n    'A little important': 4.0,\n    'Not at all important': 5.0\n}\ndf_yougov['wc_together'] = df_yougov['wc_together'].map(mapping).fillna(np.nan)\n"

In [156]:

'''counts = df_yougov['wc_together'].value_counts()
# Print the counts 
print('1: ', counts.get(1.0, 0))
print('2: ', counts.get(2.0, 0))
print('3: ', counts.get(3.0, 0))
print('4: ', counts.get(4.0, 0))
print('5: ', counts.get(5.0, 0))'''


"counts = df_yougov['wc_together'].value_counts()\n# Print the counts \nprint('1: ', counts.get(1.0, 0))\nprint('2: ', counts.get(2.0, 0))\nprint('3: ', counts.get(3.0, 0))\nprint('4: ', counts.get(4.0, 0))\nprint('5: ', counts.get(5.0, 0))"

In [157]:
### same with all vars past this point

In [158]:
# Check the data type of the column 
print(df_anes['auth_grid_2'])

0       1.0
1       1.0
2       1.0
3       2.0
4       1.0
       ... 
8275    2.0
8276    2.0
8277    1.0
8278    2.0
8279    1.0
Name: auth_grid_2, Length: 8280, dtype: float64


In [159]:
### optional recode all variables to between 0 and 1
from sklearn.preprocessing import MinMaxScaler

# return a new df that is a copy of df, with: rescale all columns to be
#  between 0 and 1, inclusive. Drop any non-numeric columns. Drop any
# rows that are missing at least one predictor.
def data_rescale(df, predictors):
    df = df.copy() # preserve immutability

    # Select only the numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns

    # drop any rows that are missing at least one predictor
    df = df.dropna(subset=predictors)
    # print('the dataframe we\'re rescaling is size: ') # debug
    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler to the data and transform it
    scaled_values = scaler.fit_transform(df[numeric_cols])

    # Create a new DataFrame with the scaled values, maintaining the original column names
    scaled_df = pd.DataFrame(scaled_values, columns=numeric_cols, index=df.index)

    return scaled_df


In [188]:
combined = [
                   #'christian_nationalism',
                   #'turnout20post', 
                   #'presvote20post',
                   #'housevote20post',
                   #'senvote20post',
                   #'trump_presidential_approval',
                   'pff_jb',
                   'pff_dt',
                   'ideo7',
                   'pid7',
                  # 'para_social_grid_2',
                   #'election_fairnness',
                   #'cab_b', w1, w2, w3
                   'educ',
                   #'race',
                   #'hispanic',
                   #'partisan_violence',
                   'immigrant_citizenship',
                   'immigrant_deport',
                   'auth_grid_1',
                   'auth_grid_3',
                   'auth_grid_2',
                   'faminc_new',
                   'abortion'
                  # 'wc_together', w1, w2, w3
                   #'wc_jobs',
                   #'racial_id',
                   #'hardworkingvlazy',
                   #'pronenot_violence',
                   #'group_disc_black',
                   #'group_disc_hispanic',
                   #'group_disc_white'
                   ] 

In [161]:
# Save the DataFrames as Stata .dta files
df_anes = data_rescale(df_anes, combined)
df_yougov = data_rescale(df_yougov, combined)

df_anes.to_stata('anes_recode_between01.dta')

c:\Users\kirin\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\kirin\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))
c:\Users\kirin\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:701: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
c:\Users\kirin\anaconda3\Lib\site-packages\sklearn\utils\_array_api.py:718: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [162]:
#df_yougov = df_yougov.drop(columns=['race'])
print(df_anes.select_dtypes(include=['number']).columns)

Index(['V201001', 'V201002a', 'V201002b', 'V201003', 'V201004', 'V201005',
       'V201006', 'V201007a', 'V201007b', 'V201007c',
       ...
       'V201658m', 'V201658n', 'V201658p', 'immigrant_citizenship',
       'immigrant_deport', 'auth_grid_1', 'auth_grid_3', 'auth_grid_2',
       'faminc_new', 'white'],
      dtype='object', length=721)


In [163]:
df_yougov.to_stata('yougov_recode_between01.dta')

In [164]:
import os
print(os.getcwd())

c:\Users\kirin\OURSIP_summer24


In [ ]:
print(df_yougov['immigrant_citizenship'])

2       1.000000
5       1.000000
6       1.000000
7       1.000000
8       1.000000
          ...   
3266    0.666667
3267    0.833333
3268    1.000000
3270    0.666667
3274    1.000000
Name: immigrant_citizenship, Length: 2050, dtype: float64


In [ ]:
#checking correlations to make sure everything looks ok

correlation_value = df_anes['ideo7'].corr(df_anes['faminc_new'])
print("Correlation between ideo7 and abortion in anes:", correlation_value)

correlation_value = df_yougov['ideo7'].corr(df_yougov['faminc_new'])
print("Correlation between ideo7 and abortion in yougov:", correlation_value)


Correlation between ideo7 and abortion in anes: 0.041372904636019586
Correlation between ideo7 and abortion in yougov: 0.07683665415237151


In [190]:
df1 = pd.read_stata("/Users/kirin/OURSIP_summer24/yougov_recode_between01.dta")
df2 = pd.read_stata("/Users/kirin/OURSIP_summer24/anes_recode_between01.dta")


for var in combined:
    correlation_value = df1['ideo7'].corr(df1[var])
    print("Correlation in anes between ideo7 and ", var, ": ", correlation_value)

    correlation_value = df2['ideo7'].corr(df2[var])
    print("Correlation in yougov between ideo7 and ", var, ": ", correlation_value)
    print()



Correlation in anes between ideo7 and  pff_jb :  0.5787371219428488
Correlation in yougov between ideo7 and  pff_jb :  0.631810067474798

Correlation in anes between ideo7 and  pff_dt :  0.6674760251161306
Correlation in yougov between ideo7 and  pff_dt :  0.7240735805026022

Correlation in anes between ideo7 and  ideo7 :  1.0
Correlation in yougov between ideo7 and  ideo7 :  1.0

Correlation in anes between ideo7 and  pid7 :  0.7112732478768897
Correlation in yougov between ideo7 and  pid7 :  0.7336009501613181

Correlation in anes between ideo7 and  educ :  0.13611467627225118
Correlation in yougov between ideo7 and  educ :  0.1177024685913317

Correlation in anes between ideo7 and  immigrant_citizenship :  -0.53330232428134
Correlation in yougov between ideo7 and  immigrant_citizenship :  -0.35812844901602636

Correlation in anes between ideo7 and  immigrant_deport :  0.5977907025105856
Correlation in yougov between ideo7 and  immigrant_deport :  0.5349287140090618

Correlation in a